In [4]:
import timm

In [5]:
import torch
import torchvision
import torchvision.transforms as T

In [6]:
import segmentation_models_pytorch as smp

In [15]:
m = timm.create_model('mobilenetv3_large_100', pretrained=True, features_only=True)
m.eval()

MobileNetV3Features(
  (conv_stem): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): Hardswish()
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
        (bn1): BatchNormAct2d(
          16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): ReLU(inplace=True)
        )
        (se): Identity()
        (conv_pw): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
          16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): Identity()
        )
        (drop_path): Identity()
      )
    )
    (1): Sequential(
      (0): InvertedResidual(
        (conv_pw): Conv2d(16

In [16]:
from PIL import Image

In [17]:
!wget https://upload.wikimedia.org/wikipedia/commons/f/fe/Giant_Panda_in_Beijing_Zoo_1.JPG -O panda.jpg
img = Image.open('panda.jpg').convert('RGB')
trans = T.Compose([T.Resize((224,224)),T.ToTensor()])
timg = trans(img).unsqueeze(0)

--2022-09-12 15:07:03--  https://upload.wikimedia.org/wikipedia/commons/f/fe/Giant_Panda_in_Beijing_Zoo_1.JPG
Resolving upload.wikimedia.org (upload.wikimedia.org)... 91.198.174.208
Connecting to upload.wikimedia.org (upload.wikimedia.org)|91.198.174.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 116068 (113K) [image/jpeg]
Saving to: 'panda.jpg'

panda.jpg           100%[===================>] 113.35K  --.-KB/s    in 0.1s    

2022-09-12 15:07:04 (761 KB/s) - 'panda.jpg' saved [116068/116068]



In [22]:
%%time
out = m(timg)
for o in out:
    print(o.shape)

torch.Size([1, 16, 112, 112])
torch.Size([1, 24, 56, 56])
torch.Size([1, 40, 28, 28])
torch.Size([1, 112, 14, 14])
torch.Size([1, 960, 7, 7])
CPU times: user 139 ms, sys: 47.3 ms, total: 187 ms
Wall time: 147 ms


In [39]:
with open('imagenet-classes.txt','r') as f:
    classes_names = [
        line.rstrip()
        for line in f
    ]

In [12]:
pred = out[0].argmax()
print(classes_names[pred])

giant panda, panda, panda bear, coon bear, Ailuropoda melanoleuca


In [13]:
rten = torch.rand(1,3,225,225, requires_grad=True)
traced_model = torch.jit.trace(m,rten)
traced_model.save('model.pt')

In [46]:
m2 = smp.FPN(
    encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=3,                      # model output channels (number of classes in your dataset)
)

In [31]:
m3 = smp.FPN(
    encoder_name="resnet50",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="swsl",         # use `imagenet` pre-trained weights for encoder initialization
)

In [49]:
m2.eval()
m3.eval()

FPN(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
       

In [54]:
%%time
out2 = m2(timg).detach()
print(out2.shape)

torch.Size([1, 3, 224, 224])
CPU times: user 456 ms, sys: 48.2 ms, total: 505 ms
Wall time: 274 ms


In [51]:
%%time
out3 = m3(timg).detach()

CPU times: user 1.25 s, sys: 86.6 ms, total: 1.34 s
Wall time: 704 ms


In [52]:
import numpy as np

In [55]:
pred2 = torch.softmax(out2, dim=1)[0][0]
mash = (pred2.cpu().numpy()*255).astype(np.uint8)
Image.fromarray(mash).save('out.png')

In [57]:
rten = torch.rand(1,3,224,224, requires_grad=True)
traced_model = torch.jit.trace(m2,rten)
traced_model.save('model2.pt')

/opt/anaconda3/lib/python3.9/site-packages/torch/nn/functional.py:2515: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  _verify_batch_size([input.size(0) * input.size(1) // num_groups, num_groups] + list(input.size()[2:]))
